In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import joblib
emb_matrix_train = joblib.load("/content/drive/MyDrive/absita_comfort_embedd_128_768_alberto_train.joblib")
wmal_matrix_train = joblib.load("/content/drive/MyDrive/wmal_train_comfort_128.joblib")

In [2]:
import pandas as pd
dataset_DF = pd.read_csv("/content/drive/MyDrive/comfort_train_processed.csv")

In [3]:
from sklearn.preprocessing import OneHotEncoder
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example
data = dataset_DF["pos"]
values = array(data)
print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
print(inverted)

[0 1 1 ... 1 0 0]
[0 1 1 ... 1 0 0]
[[1. 0.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [1. 0.]
 [1. 0.]]
[0]


In [4]:
from keras.layers import Input, Dense, LSTM, Bidirectional, TimeDistributed, Flatten, Concatenate, Dropout
!pip install keras-self-attention
from keras_self_attention import SeqWeightedAttention, SeqSelfAttention
from keras.layers import Add, Multiply

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
emb_matrix_test = joblib.load("/content/drive/MyDrive/absita_comfort_embedd_128_768_alberto_test.joblib")
dataset_test = pd.read_csv("/content/drive/MyDrive/comfort_test_processed.csv")
wmal_matrix_test = joblib.load("/content/drive/MyDrive/wmal_test_comfort_128.joblib")

In [6]:
# define example
data = dataset_test["pos"]
values = array(data)
print(values)
# integer encode
integer_encoded_test = label_encoder.transform(values)
print(integer_encoded_test)
# binary encode
integer_encoded_test = integer_encoded_test.reshape(len(integer_encoded_test), 1)
onehot_encoded_test = onehot_encoder.transform(integer_encoded_test)
print(onehot_encoded_test)

[0 0 0 ... 1 1 0]
[0 0 0 ... 1 1 0]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [1. 0.]]


In [7]:
import tensorflow.keras as keras
from sklearn.metrics import precision_recall_fscore_support as score
import numpy as np

input_emb = Input(shape=(128,768))
#input_wmal = Input(shape=(128,))

#Bi-LSTM
#bi = Bidirectional(LSTM(64, return_sequences=True))(input_emb)
bi = LSTM(64, return_sequences=True)(input_emb)
att = SeqSelfAttention(attention_width=15,attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,
                       attention_activation="sigmoid",
                       kernel_regularizer=keras.regularizers.l2(1e-8),#(1e-6),
                       bias_regularizer=keras.regularizers.l1(1e-4),#(1e-4),
                       attention_regularizer_weight=1e-8,#1e-4,
                       use_attention_bias=True) (bi)

app = Add() ([bi, att])                      
flat = Flatten()(app)
den1 = Dense(4000) (flat)
den2 = Dense(2000) (den1)
den3 = Dense(1000) (den2)
den4 = Dense(500) (den3)
den5 = Dense(256) (den4)
den6 = Dense(128) (den5)
den7 = Dense(64) (den6)

#WMAL 
#d1 = Dense(64) (input_wmal)

#Concat
#concat = Concatenate(axis=1)([den7, d1])
d1 = Dense(32, activation="relu") (den7)
output = Dense(2,activation="softmax") (d1)

m = keras.Model(inputs=[input_emb], outputs=output)

opt = keras.optimizers.RMSprop(lr=0.0001)

m.compile(loss="categorical_crossentropy",optimizer=opt,metrics=["accuracy"])

class myCallback(keras.callbacks.Callback):
    bestF1=0
    def __init__(self):
        super().__init__()
        self._get_pred = None
        self.preds = []
    def on_epoch_end(self,epoch, logs=None):
        predictions = m.predict([emb_matrix_test])
        predictions_vec = np.zeros((1171,4))
        i = 0
        confidences= []
        for item in predictions:
          index = np.argmax(item);
          confidences.append(item[index])
          predictions_vec[i][index] = 1
          i = i+1
        labels_to_eval = []
        for vec in predictions_vec:
          inverted = label_encoder.inverse_transform([argmax(vec)])[0]
          labels_to_eval.append(inverted)
        
        precision,recall,fscore,support=score(dataset_test["pos"],labels_to_eval,average='macro')
        print("Test f1-score:"+str(fscore))
        if fscore > self.bestF1:
          m.save("bestModel.h5")
          print("BEST MODEL!!!!!")
          self.bestF1 = fscore

callbacks = [myCallback()]

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
m.summary()

In [8]:
m.fit([emb_matrix_train],onehot_encoded,batch_size=128, epochs=40,validation_split=0.15, callbacks=[callbacks])
#m.load_weights("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/models/model_3_sentipolc/model_3_neg_sentipolc_0.72803.h5")

Epoch 1/40
18/18 [==============================] - ETA: 0s - loss: 1.3284 - accuracy: 0.7400Test f1-score:0.7711316087326868
BEST MODEL!!!!!
18/18 [==============================] - 17s 268ms/step - loss: 1.3284 - accuracy: 0.7400 - val_loss: 0.5157 - val_accuracy: 0.7706
Epoch 2/40
18/18 [==============================] - ETA: 0s - loss: 0.3386 - accuracy: 0.8814Test f1-score:0.8112173475990285
BEST MODEL!!!!!
18/18 [==============================] - 4s 212ms/step - loss: 0.3386 - accuracy: 0.8814 - val_loss: 0.5271 - val_accuracy: 0.7706
Epoch 3/40
18/18 [==============================] - ETA: 0s - loss: 0.2414 - accuracy: 0.9097Test f1-score:0.832527048874518
BEST MODEL!!!!!
18/18 [==============================] - 3s 192ms/step - loss: 0.2414 - accuracy: 0.9097 - val_loss: 0.3521 - val_accuracy: 0.8703
Epoch 4/40
18/18 [==============================] - ETA: 0s - loss: 0.1956 - accuracy: 0.9339Test f1-score:0.8502125202378472
BEST MODEL!!!!!
18/18 [==============================] 

In [9]:
m.load_weights("bestModel.h5")
predictions = m.predict([emb_matrix_test])

In [10]:
import numpy as np
predictions_vec = np.zeros((1171,4))
i = 0
confidences= []
for item in predictions:
  index = np.argmax(item);
  confidences.append(item[index])
  predictions_vec[i][index] = 1
  i = i+1
labels_to_eval = []
for vec in predictions_vec:
  inverted = label_encoder.inverse_transform([argmax(vec)])[0]
  labels_to_eval.append(inverted)

In [11]:
!pip install scikit-learn

from sklearn.metrics import classification_report
print(classification_report(dataset_test["pos"],labels_to_eval,digits=5))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
              precision    recall  f1-score   support

           0    0.90043   0.94260   0.92103       662
           1    0.92050   0.86444   0.89159       509

    accuracy                        0.90863      1171
   macro avg    0.91047   0.90352   0.90631      1171
weighted avg    0.90916   0.90863   0.90824      1171



In [12]:
m.save("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/models/model4_noWMAL_pos.h5")

In [ ]:
dataset_test.to_csv("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/models/predictions_creagritrend_neg_full_0.72649_noWMAL.csv");